# Monterey Flipper Pinball Standings Projections

## Purpose
This notebook is used to project the final standings of the Monterey Flipper Pinball league.
The highest 5 weekly scores for each player are summed at the end of the season to produce a winner.
There are 10 weeks total, so you can drop up to 5 scores if you attend every week.

As of right now it is the end of week 4, my initial idea is to take your average score and use that to replace any lower scores, as well as the correct number of future weeks.

## TODO/Issues
- Possibly redo converting the json to a dataframe, there has to be a better way
- Projections fail if the average is taken over a player's best games
- API fails to pull all players in Winter 17 MFP league, (series_id = 475)

In [88]:
import pandas as pd
import numpy as np
import requests
import json
from collections import defaultdict

In [89]:
# MatchPlay API with the Series ID for MFP Spring 18
series_id = '582'
series_url = 'https://matchplay.events/data/series/'+series_id+'/standings'

In [90]:
# Call the API
r = requests.get(series_url)
data = r.json()

In [91]:
# Split up the response, which gives overall standings and also weekly scores (tournament_points)
overall = data['overall']
overall = pd.DataFrame.from_records(overall)
tournament_points = data['tournament_points']

In [92]:
# Janky code to rearrange the tournament_points json response

points = defaultdict(list)
num_weeks = 0
for week in tournament_points:
    if week['status'] != 'planned':
        num_weeks += 1
        for player in week['points'].items():
            points[(player[0],week['tournament_id'])].append(player[1])


In [93]:
# Changing the points dict into a tidy dataframe

points_df = pd.DataFrame(points)
points_df = points_df.transpose().reset_index(1).pivot(columns='level_1',values=0)

In [94]:
# Url to grab weekly standings for names
week_ids = points_df.columns[:num_weeks]
tournament_url_a = 'https://matchplay.events/data/tournaments/'
tournament_url_b = '/standings'

In [95]:
player_names = {}
for week_id in week_ids:
    tournament_url = tournament_url_a + str(week_id) + tournament_url_b
    r = requests.get(tournament_url)
    data = r.json()
    for player in data:
        if player['player_id'] not in player_names:
            player_names[str(player['player_id'])] = player['name']

In [96]:
points_df.index = points_df.index.to_series().map(player_names)

In [97]:
# Apply function that takes the row and does necessary computations for extra columns
weeks_left = 10-num_weeks

def compute_projections(row):
    # Compute average score of all weeks
    row['avg_score'] = row[:num_weeks].mean()
    
    # Number of weeks played
    row['num_games'] = row[:num_weeks].count()
    
    # Total number of points accumulated
    row['cur_total'] = sum([points for points in row[:num_weeks] if points > 0])

    # Find best five games for a player
    best_games = [points for points in row[:num_weeks] if points > 0]
    best_games.sort(reverse=True)
    best_games = best_games[:5]
    
    # Find a player's current score, the metric by which players are judged at the end of the season
    row['cur_score'] = sum(best_games)
    
    # Compute average score of a player's best 5 weeks
    #row['avg_score'] = row['cur_score']/len(best_games)
    
    # Exclude a players
    best_games_aa = [points for points in best_games if points >= row['avg_score']]
    best_games_aa.sort(reverse=True)    
    
    # mean_adj_score: the points added to a player's projection to supplant their below average games
    # initialize variable to 0 for players that have 5 games above their average
    #     this may be deprecated by finding the average of the 5 best games instead of all games
    row['mean_adj_score'] = 0
    
    # If a player has five games above their average, no games are replaced by that average
    if len(best_games_aa) >= 5:
        row['kept_score'] = sum(best_games_aa[:5])
        row['proj_score'] = row['kept_score']
        
    # Replace weeks below average with the average score, depending on how many weeks are left
    if len(best_games_aa) < 5:
        row['kept_score'] = sum(best_games_aa)
        
        # replace the weeks below average to fill to 5, or however many weeks are left if it wouldn't be possible
        if (5-len(best_games_aa) <= weeks_left):
            row['mean_adj_score'] = (5-len(best_games_aa))*row['avg_score']
        else:
            row['kept_score'] = sum(best_games[:-weeks_left])
            row['mean_adj_score'] = weeks_left*row['avg_score']
        
        # Project final score based on player's average
        row['proj_score'] = row['kept_score'] + row['mean_adj_score']
    
    # Find a player's maximum possible score assuming they score a perfect 35 every remaining week
    if weeks_left < 5:
        row['max_score_possible'] = sum(best_games[:5-weeks_left]) + weeks_left*35
    else:
        row['max_score_possible'] = 5*35
    
    # The number of weeks of average score added to a player's projection,
    #   so the number of weeks the player would need to do above average to reach their projection
    row['remaining_weeks_above_avg'] = (row['mean_adj_score']/row['avg_score'])
    
    return row


In [98]:
# Apply the calculations
final_points_df = points_df.apply(compute_projections, axis=1)

In [99]:
current_leader = final_points_df.cur_score.max()

# calculate the average a player must get in the remaining games to beat the leader
def compute_required_avg(row):
    
    
    return row

In [100]:

final_points_df = final_points_df.apply(compute_required_avg, axis=1)

In [101]:
final_points_df.sort_values('proj_score', ascending = False)

level_1,17082,17083,17084,17085,17086,17087,17088,avg_score,num_games,cur_total,cur_score,mean_adj_score,kept_score,proj_score,max_score_possible,remaining_weeks_above_avg
Cary Carmichael,21.0,14.0,27.0,33.0,29.0,27.0,21.0,24.571429,7.0,172.0,137.0,24.571429,116.0,140.571429,167.0,1.0
Matthew Talley,21.0,27.0,27.0,NaN,23.0,29.0,27.0,25.666667,6.0,154.0,133.0,25.666667,110.0,135.666667,161.0,1.0
Mike Hubbard,NaN,27.0,20.0,22.0,21.0,31.0,19.0,23.333333,6.0,140.0,121.0,70.000000,58.0,128.000000,163.0,3.0
Greg Friedman,33.0,17.0,NaN,23.0,25.0,NaN,15.0,22.600000,5.0,113.0,113.0,45.200000,81.0,126.200000,163.0,2.0
Fred Hamilton,21.0,18.0,27.0,25.0,21.0,23.0,27.0,23.142857,7.0,162.0,123.0,46.285714,79.0,125.285714,159.0,2.0
Tony Lavigna,25.0,19.0,NaN,17.0,NaN,25.0,29.0,23.000000,5.0,115.0,115.0,46.000000,79.0,125.000000,159.0,2.0
John Tracey,23.0,25.0,25.0,27.0,19.0,23.0,21.0,23.285714,7.0,163.0,123.0,46.571429,77.0,123.571429,157.0,2.0
Ernie Deakyne,25.0,23.0,21.0,26.0,23.0,14.0,25.0,22.428571,7.0,157.0,122.0,0.000000,122.0,122.000000,156.0,0.0
Quinn Tomlinson,19.0,25.0,25.0,21.0,NaN,25.0,23.0,23.000000,6.0,138.0,119.0,23.000000,98.0,121.000000,155.0,1.0
Nicole Carmichael,23.0,NaN,23.0,20.0,29.0,19.0,21.0,22.500000,6.0,135.0,116.0,45.000000,75.0,120.000000,157.0,2.0
